<a href="https://colab.research.google.com/github/keithfernandez0/AI-projects/blob/main/src/federated_learning/tensorflow_test_environment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Federated Learning Research Project: Kean University, Spring 2024
## Authors: Keith Fernandez, Dr. Bin Hu, Dr. Daehan Kwak
### Links:


1.   [Weekly Meeting Schedule & Summary Document](https://docs.google.com/document/d/1YWZ8bfra6xjUQdf0sjmYMRDVvtZvgLAmkiINDndAP5k/edit#heading=h.rzvzrn7tc2q7)
2.   [Literature Review Packet](https://)
3.   [Codebase Testing / Compression Planning Review Packet](https://docs.google.com/document/d/1gWcF75IraJbcPPBJYYnoGve5uyfv3jo_yVFQHJbi0a0/edit#heading=h.1sbkr0ejpc52)

---

*   Prelimaries: Discuss time scale and end goal. Wishing to be published in a localized journal / upgrading from conferences.
*   Week 1: Literature review, discussion of papers, overview
*   Week 2: Finding and discussing potential codebases to be used, what compression / pruning algorithms are best.
*   Week 3: Testing of coding environment, verifying modules run in Colab notebook.



# TO-DO


- [x] Create Google Colab notebook.
- [x] Have Google Colab runtime default to external TPU/GPU.
- [x] Provide basic synopsis / links to planning sheets.
- [x] Credit project authors / research advisors.
- [x] Sync project to [GitHub repository.](https://github.com/keithfernandez0/AI-projects/blob/main/src/federated_learning/tensorflow_test_environment.ipynb)
  - [x] Authenticate GitHub credentials on local machine with `git-cli`
  - [x] Verify remote push / git commits are working.
- Import relevant libraries:
  - [x] TensorFlow
  - [x] TensorFlow Federated
  - [x] TensorFlow Model Optimization ToolKit
  - Microsoft Model Compression ToolKit
- Create simple test network (3x4 depth)




In [ ]:
!pip install --upgrade tensorflow tensorflow_federated tensorflow_model_optimization tf_keras matplotlib numpy

import collections
import tf_keras as keras
import tensorflow as tf
import tensorflow_federated as tff
import tensorflow_model_optimization as tfmot

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
!pip show tensorflow
!pip show tensorflow_federated
!python3 --version

## Verification of modules working

The following tests will be done to individually verify each TensorFlow package on a standalone basis.


*   `tensorflow` by itself
*   `tensorflow` with `tensorflow_federated`
*   `tensorflow` with `tensorflow_model_optimization`

We will verify these packages work individually and then move on to combining them to a larger scale network for our research.



In [16]:
# ==============================================================================
# Testing base TensorFlow
# STATUS: VERIFIED
# ==============================================================================

def create_base_tf_network():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(10, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return model

# Create an instance of the test network, print summary

base_tf_network = create_base_tf_network()
base_tf_network.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 10)                7850      
                                                                 
 dense_7 (Dense)             (None, 10)                110       
                                                                 
 dense_8 (Dense)             (None, 10)                110       
                                                                 
Total params: 8070 (31.52 KB)
Trainable params: 8070 (31.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
# ==============================================================================
# Testing base TensorFlow Federated
# STATUS: VERIFIED
# ==============================================================================

def create_tf_federated_network():
    input_spec = [
        (tf.TensorSpec(dtype=tf.float32, shape=(None, 784)), tf.TensorSpec(dtype=tf.int32, shape=(None,)))
    ]
    output_spec = tff.TensorType(np.int32, shape=(None,))

    def model_fn():
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(10, activation='relu', input_shape=(784,)),
            tf.keras.layers.Dense(10, activation='relu'),
            tf.keras.layers.Dense(10, activation='softmax')
        ])

        return tff.learning.models.from_keras_model(
            model,
            input_spec=input_spec[0],
            loss=tf.keras.losses.SparseCategoricalCrossentropy(),
            metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
        )

    # Define the client and server optimizers
    client_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate=0.1)
    server_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate=1.0)

    # Build the federated averaging process
    return tff.learning.algorithms.build_weighted_fed_avg(
        model_fn,
        client_optimizer_fn=client_optimizer_fn,
        server_optimizer_fn=server_optimizer_fn
    )

# Create an instance of the test network
tf_federated_network = create_tf_federated_network()
print(tf_federated_network)


In [20]:
# ==============================================================================
# Testing base TensorFlow Model Optimization Toolkit
# STATUS: VERIFIED
# ==============================================================================

def create_tf_model_optimization_network():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(10, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return model

# Create an instance of the test network
tf_model_optimization_network = create_tf_model_optimization_network()

# Prune the model
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                              final_sparsity=0.80,
                                                              begin_step=2000,
                                                              end_step=4000)
}

pruned_model = tfmot.sparsity.keras.prune_low_magnitude(tf_model_optimization_network, **pruning_params)

pruned_model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_dense_  (None, 10)                15692     
 9 (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_dense_  (None, 10)                212       
 10 (PruneLowMagnitude)                                          
                                                                 
 prune_low_magnitude_dense_  (None, 10)                212       
 11 (PruneLowMagnitude)                                          
                                                                 
Total params: 16116 (62.96 KB)
Trainable params: 8070 (31.52 KB)
Non-trainable params: 8046 (31.44 KB)
_________________________________________________________________
